# Tutorial 03: weak imposition of Dirichlet BCs by a Lagrange multiplier (linear problem)

In this tutorial we solve the problem

$$\begin{cases}
-\Delta u = f, & \text{in } \Omega,\\
 u   = g, & \text{on } \Gamma = \partial\Omega,
\end{cases}$$

where $\Omega$ is the unit ball in 2D.

We compare the following two cases:
* **strong imposition of Dirichlet BCs**:
the corresponding weak formulation is
$$
\text{find } u \in V_g \text{ s.t. } \int_\Omega \nabla u \cdot \nabla v = \int_\Omega f v, \quad \forall v \in V_0\\
$$
where
$$
V_g = \{v \in H^1(\Omega): v|_\Gamma = g\},\\
V_0 = \{v \in H^1(\Omega): v|_\Gamma = 0\}.\\
$$
* **weak imposition of Dirichlet BCs**: this requires an introduction of a multiplier $\lambda$ which is restricted to $\Gamma$, and solves
$$
\text{find } w, \lambda \in V \times M \text{ s.t. }\\
\begin{cases}
\int_\Omega \nabla w \cdot \nabla v + \int_\Gamma \lambda v = \int_\Omega f v, & \forall v \in V,\\
\int_\Gamma w \mu = \int_\Gamma g \mu, & \forall \mu \in M
\end{cases}
$$
where
$$
V = H^1(\Omega),\\
M = L^{2}(\Gamma).\\
$$

This example is a prototypical case of problems containing subdomain/boundary restricted variables (the Lagrange multiplier, in this case).

In [ ]:
import numpy as np
from mpi4py import MPI
from petsc4py import PETSc
from ufl import grad, inner, Measure, TestFunction, TrialFunction
from dolfinx.fem import DirichletBC, Function, FunctionSpace, LinearProblem, locate_dofs_topological
from dolfinx.io import XDMFFile
from dolfinx.plot import create_vtk_topology
from multiphenicsx.fem import (assemble_matrix_block, assemble_scalar, assemble_vector_block,
                               BlockVecSubVectorWrapper, create_vector_block, DofMapRestriction)
import pyvista

### Mesh

In [ ]:
with XDMFFile(MPI.COMM_WORLD, "data/circle.xdmf", "r") as infile:
    mesh = infile.read_mesh()
    subdomains = infile.read_meshtags(mesh, name="subdomains")
    mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)
    boundaries = infile.read_meshtags(mesh, name="boundaries")
facets_Gamma = boundaries.indices[boundaries.values == 1]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

In [ ]:
def dolfinx_to_pyvista_mesh(mesh):
    num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
    cell_entities = np.arange(num_cells, dtype=np.int32)
    pyvista_cells, cell_types = create_vtk_topology(mesh, mesh.topology.dim, cell_entities)
    grid = pyvista.UnstructuredGrid(pyvista_cells, cell_types, mesh.geometry.x)
    return grid

In [ ]:
def pyvista_mesh_plot(mesh):
    grid = dolfinx_to_pyvista_mesh(mesh)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_mesh_plot(mesh)

### Weak imposition of Dirichlet BCs

In [ ]:
# Define a function space
V = FunctionSpace(mesh, ("Lagrange", 2))
M = V.clone()

In [ ]:
# Define restrictions
dofs_V = np.arange(0, V.dofmap.index_map.size_local + V.dofmap.index_map.num_ghosts)
dofs_M_Gamma = locate_dofs_topological(M, boundaries.dim, facets_Gamma)
restriction_V = DofMapRestriction(V.dofmap, dofs_V)
restriction_M_Gamma = DofMapRestriction(M.dofmap, dofs_M_Gamma)
restriction = [restriction_V, restriction_M_Gamma]

In [ ]:
# Define trial and test functions
(u, l) = (TrialFunction(V), TrialFunction(M))
(v, m) = (TestFunction(V), TestFunction(M))

In [ ]:
# Define problem block forms
g = Function(V)
g.interpolate(lambda x: np.sin(3 * x[0] + 1) * np.sin(3 * x[1] + 1))
a = [[inner(grad(u), grad(v)) * dx, l * v * ds],
     [u * m * ds, None]]
f = [v * dx, g * m * ds]

In [ ]:
# Assemble the block linear system
A = assemble_matrix_block(a, bcs=[], restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=[], restriction=restriction)

In [ ]:
# Solve
ul = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.comm)
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, ul)
ul.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
# Split the block solution in components
(u, l) = (Function(V), Function(M))
with BlockVecSubVectorWrapper(ul, [V.dofmap, M.dofmap], restriction) as ul_wrapper:
    for ul_wrapper_local, component in zip(ul_wrapper, (u, l)):
        with component.vector.localForm() as component_local:
            component_local[:] = ul_wrapper_local

In [ ]:
def pyvista_scalar_field_plot(mesh, scalar_field, name):
    grid = dolfinx_to_pyvista_mesh(mesh)
    grid.point_arrays[name] = scalar_field.compute_point_values()
    grid.set_active_scalars(name)
    plotter = pyvista.PlotterITK()
    plotter.add_mesh(grid)
    plotter.show()

In [ ]:
pyvista_scalar_field_plot(mesh, u, "u")

In [ ]:
pyvista_scalar_field_plot(mesh, l, "l")

### Strong imposition of Dirichlet BCs

In [ ]:
# Define Dirichlet BC object on Gamma
dofs_V_Gamma = locate_dofs_topological(V, boundaries.dim, facets_Gamma)
bc_ex = DirichletBC(g, dofs_V_Gamma)

In [ ]:
# Solve
u_ex = Function(V)
problem_ex = LinearProblem(
    a[0][0], f[0], bcs=[bc_ex], u=u_ex,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
problem_ex.solve()
u_ex.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
pyvista_scalar_field_plot(mesh, u_ex, "u")

### Comparison and error computation

In [ ]:
u_ex_norm = np.sqrt(mesh.comm.allreduce(assemble_scalar(inner(grad(u_ex), grad(u_ex)) * dx), op=MPI.SUM))
err_norm = np.sqrt(mesh.comm.allreduce(assemble_scalar(inner(grad(u_ex - u), grad(u_ex - u)) * dx), op=MPI.SUM))
print("Relative error is equal to", err_norm / u_ex_norm)
assert np.isclose(err_norm / u_ex_norm, 0., atol=1.e-10)